# Chapter 21 – DNA Sequence Analysis of MinION Nanopore Reads
Our project is dedicated to analyzing sequence reads from a plasmid following its digestion by either a restriction enzyme or a Cas9/gRNA complex. The aim is the identification of cutting sites within the plasmid. The reads were generated by DNA sequencing with a MinION, a pocket-sized sequencing device based on Nanopore technology. For the project, the FASTA5 read files are provided. The binary FASTA5 format standardizes nucleotide sequence data storage, ensuring compatibility across various bioinformatics tools. For the analysis, a suite of specialized tools is employed. The Guppy tool facilitates base calling and demultiplexing of nanopore sequence reads based on barcode identifiers. qfilter, a little tool developed in my lab, is utilized to filter sequencing data, retaining only high-quality reads while discarding noisy or low-quality data. The Minimap tool aligns long nucleotide sequences to a reference genome, facilitating the detection of similarities and diﬀerences relevant to cutting sites. This in turn is done with the Integrated Genome Viewer (IGV). Overall, the combined use of these tools enables the processing and analysis of MinION-derived sequencing data, ultimately leading to the identification of sequence features. Finally, the simulation of nanopore sequence reads is exemplified. 

> If you read these instructions on **GitHub**, you will notice that some lines of code are incomplete at the end. Nevertheless, you can copy the cells completely into the clipboard by double-clicking on them.

Necessary Installations:

- `sudo apt install -y hdfview hdf5-tools ont-fast5-api dotter porechop minimap2 samtools igv`
- `wget https://cdn.oxfordnanoportal.com/software/analysis/ont_guppy_cpu_6.3.7-1~focal_amd64.deb`
- `sudo apt install -f ./ont_guppy_cpu_6.3.7-1~focal_amd64.deb`



In [ ]:
pwd

### Download Data Files

In [ ]:
wget https://www.dropbox.com/s/si08bjwdvcmtuls/ALT327_c9ffe0f3_0.fast5

In [ ]:
wget https://www.dropbox.com/s/88hqohg3271un21/pBR322.fasta

In [ ]:
ls

### Move Files to new Folder

In [ ]:
mkdir OriginalFast5

In [ ]:
cp ALT327_c9ffe0f3_0.fast5 OriginalFast5

### Viewing the FAST5 Files
FAST5 files are Hierarchical Data Format 5 (HDF5) files with a specific structure defined by Oxford Nanopore Technologies (ONT) for the storage of raw sequencing data and are generated by, for example, MinION. There are two types of FAST5: Single-FAST5 and Multi-FAST5. A multi-FAST5 file contains data for several reads in a single file, while a single-FAST5 file contains data from a single read. Single FAST5 files are rarely used.
A free program for viewing the files is HDFView, which you can download for all operating systems at [hdfgroup.org](https://hdfgroup.org). We have already installed it above with APT.


You can start the graphical version with `hdfview` in the terminal.

However, we use command line tools here.

The program `h5ls` is available for this purpose, which lists the contents of a FAST5 file. As the MinION file is a multi-FAST5 file, we limit the output to three lines with the command `head -3`.

In [ ]:
h5ls OriginalFast5/ALT327_c9ffe0f3_0.fast5 | head -3

To check what is stored for a single read, you can specify this read with `h5ls` like a directory, i.e. separated with a slash:


In [ ]:
h5ls OriginalFast5/ALT327_c9ffe0f3_0.fast5/read_0048d23d-4b49-476b-b18c-d102e7a3e393

In [ ]:
h5ls OriginalFast5/ALT327_c9ffe0f3_0.fast5/read_0048d23d-4b49-476b-b18c-d102e7a3e393/Raw/Signal

With the `--data` option we can display individual data records. A “subdirectory” contains the raw data from the MinION, for example the voltage changes across a pore (squiggle plot):

In [ ]:
h5ls -d OriginalFast5/ALT327_c9ffe0f3_0.fast5/read_0048d23d-4b49-476b-b18c-d102e7a3e393/Raw/Signal | head -3

**Oops**: *"Unable to print data."* Something went wrong!?! Oxford Nanopore uses two different methods to compress the raw data in the FAST5 files: vbz or gzip. Recently, ONT's proprietary VBZ compression has been used, which older analysis programs sometimes have problems with. The program suite `ont-fast5-api` can be used to convert the compression and thus solve the problem of unreadable data (see above). We have already completed the installation above.

The data compression is then converted, where `--input_path OriginalFast5` names the directory with the FAST5 files and we use decompressed to name the output directory. Then the data extraction works:

In [ ]:
compress_fast5 --input_path OriginalFast5 --save_path OriginalFast5Gzip --compression gzip

In [ ]:
h5ls -d OriginalFast5Gzip/ALT327_c9ffe0f3_0.fast5/read_0048d23d-4b49-476b-b18c-d102e7a3e393/Raw/Signal | head -3

**Tip**: To speed up data analysis, the thousand reads in the FAST5 file can be separated into individual FAST5 files. This is also possible with the program suite `ont-fast5-api`:

In [ ]:
multi_to_single_fast5 --input OriginalFast5Gzip --save OriginalFast5GzipSingle

*OriginalFast5Gzip* denotes the directory with the FAST5 files and *OriginalFast5GzipSingle* denotes the output directory. FAST5 files can then be created with, for example, one hundred reads each:

In [ ]:
single_to_multi_fast5 --input OriginalFast5GzipSingle/0 --save OriginalFast5GzipMulti100 --batch_size 100

The `--save` option denotes the output directory and the `--batch_size` option denotes the number of reads per file, each of which is named *batch_0.fast5*, *batch_1.fast5*, *batch_2.fast5*, etc.

### Basecalling with Guppy
Guppy is a program that includes Oxford Nanopore Technologies' basecalling algorithms and several post-processing bioinformatics applications. This also includes demultiplexing, adapter trimming and alignments with target sequences, such as the plasmid pBR322. It is available for Windows, MacOS and Linux operating systems from Oxford Nanopore. The installation is described above. In addition to the path to a directory with our FAST5 file(s) (option `--input_path`), the basecaller needs the name of an output directory (`--save_path`) and the flow cell (`--flowcell`) and sequencing kit used - (`--kit`) combination, in our case the Flongle (FLO-FLG001) and the Rapid Barcoding Kit (SQK-RBK004). The output directory will be created automatically if it does not already exist.

On my Linux virtual machine in VitualBox on a MacBook Pro, basecalling a thousand sequences takes around an hour with one CPU (3,478,643 ms is around 58 min).


In [ ]:
guppy_basecaller --input_path OriginalFast5 --flowcell FLO-FLG001 --kit SQK-RBK004 --save_path BasecalledAll

### A first analysis with yolk
In the *basecalled/pass* directory there is a file in FASTQ format that contains the DNA sequences of the reads. Actually, we can only really talk about reads now. We will now convert this file into a file in FASTA format. To do this, we use the AWK programming language, which can be used to easily format text. Here we instruct awk to output every fourth line from line 1 or line 2. With `head -10` we limit the output to ten lines.

In [ ]:
awk 'NR%4==1 {print ">"$1} NR%4==2 {print $0}' BasecalledAll/pass/fastq_runid_c9ffe0f31e537612d2cbf4c0f7e050d9b0d928a0_0_0.fastq | head -10 > rohreads10.fasta

Dotter is a graphical dotplot program for the detailed comparison of two DNA sequences. This involves comparing every nucleotide in one sequence with every nucleotide in the other sequence. The first sequence is along the X-axis and the second sequence (or more) is along the Y-axis. In areas where the two sequences are similar, a diagonal runs through the matrix. If you were to compare a sequence to itself, you would get an angle bisector. The program call is followed by the FASTA files of the reference and the sequences to be compared.

You start Dotter with `dotter pBR322.fasta rohreads10.fasta` in the command line. The terminal is now no longer ready for input - the input prompt ($) only appears again after Dotter is closed. We receive a dot plot in a graphical window, which gives us a first impression of our result.

![dotter](dotter.png))

The sequence of the plasmid is plotted on the x-axis and the sequence of the reads is plotted on the y-axes, separated by green lines. If nucleotides on the X and Y axes match, then a point is drawn; hence the name dotplot.

### Demultiplexing with PoreChop
When different experiments are carried out, each separated by barcodes and sequenced on a flow cell, all reads end up in a FAST5 or FASTQ file. These can then be separated from each other using the `porechop` program.

In [ ]:
porechop --input BasecalledAll/pass/fastq_runid_c9ffe0f31e537612d2cbf4c0f7e050d9b0d928a0_0_0.fastq --barcode_dir BarCodesAll

The `--barcode_dir` option specifies the directory in which the found barcodes are saved. The entire issue is quite long. The last lines are crucial and show how many reads could be assigned to barcodes.

### ALTERNATIVE with Guppy
Guppy also has its own demultiplexer. Let us use it as well.

In [ ]:
guppy_barcoder --input_path BasecalledAll/pass --save_path BarcodesAll --barcode_kits SQK-RBK004

Let us count the number of demultiplexed reads:

In [ ]:
grep -c "^@" BarcodesAll/barcode0?/*.fastq

### Quality control and filtering of reads
The quality of the reads produced by the MinION will vary. If the quality is too poor, a match to the plasmid sequence cannot be detected later. The quality is indicated with the so-called PHRED quality score (or simply quality score). It has a logarithmic relationship to the base call error probabilities in base calling. A quality score of 10 means that 90% of the bases are correct; with a value of 20 it is 99%.
In addition to quality, we also filter based on read length. When the sequencing library is created, DNA molecules can attach to one another: so-called concatemers are created.
 
First we download the qFilter program and make it executable:

In [ ]:
wget "https://github.com/awkologist/qfilter/raw/main/qfilter"

In [ ]:
chmod u+x qfilter

We then filter the reads. Here we use a for loop with which all three files *BC01.fastq*, *BC02.fastq* and *BC03.fastq* are processed one after the other. The for loop has the format: `for` variable `in` list`; do` command(s)`; done` We use *i* as a variable, which takes on the values one to three one after the other. The variable value is called up again with *$i*.

With `-m` we specify the minimum average PHRED score of a read, with `-x` the minimum read length and with `-y` the maximum read length.

In [ ]:
for i in 1 2 3; do ./qfilter -m 10 -x 1000 -y 4361 BarCodesAll/BC0$i.fastq > bc0$i.all.fastq ;done

In [ ]:
wc -l bc0[123].all.fastq

### ALTERNATIVE with Guppy Data

In [ ]:
for i in 1 2 3; do ./qfilter -m 10 -x 1000 -y 4361 BarcodesAll/barcode0$i/*.fastq > bc0$i.all.fastq ;done

In [ ]:
grep -c "^@" bc0[123].all.fastq

### Mapping the reads to the plasmid
We use the Minimap program to map the reads to the plasmid pBR322. “Mapping” refers to the process that looks at which positions in the plasmid sequence the reads generated by the MinION fit. A sequence alignment is therefore carried out with each read. Here again we use a for loop, in this case with file names as a list; The question mark stands as a joker for any character.

In [ ]:
for i in bc0?.all.fastq; do minimap2 -a pBR322.fasta $i > $i.mapped.sam; done

The `-a` option tells Minimap2 to output the result in the so-called SAM format. Normally the output would be to the screen. I cropped the output again. With the redirection `>` we redirect the data to a file that we name *mapped.sam*.

### ALTERNATIVE with Guppy Data

In [ ]:
for i in bc0?.all.guppy.fastq; do minimap2 -a pBR322.fasta $i > $i.mapped.guppy.sam; done

### Preparation for Visualization with IGV
The Integrated Genome Viewer (IGV) is a program for viewing the mapped reads. To do this, the reference is first loaded (genome) and then the reads (track). In our case, the genome is the plasmid pBR322 in the file *pBR322.fasta* and the mapped reads are in the file *mapped.sam* previously created using Mminimap2. This still needs to be sorted, indexed and converted into binary format for IGV. We do this with the program `samtools`. Again we use the For loop to sort all three files, convert them to binary BAM format and then index them.

In [ ]:
for i in bc0?.all.fastq.mapped.sam; do samtools sort $i -o $i.sorted.bam; samtools index $i.sorted.bam; done

The result is the sorted files *bc0[123].all.fastq.mapped.sorted.bam* with their index in the file *bc0[123].all.fastq.mapped.sorted.bam.bai*, the IGV in background will load. With "[123]" I want to indicate that there are three files each.
The plasmid (genome) automatically indexes IGV when loading :-)

### ALTERNATIVE with Guppy Data

In [ ]:
for i in bc0?.all.guppy.fastq.mapped.guppy.sam; do samtools sort $i -o $i.sorted.guppy.bam; samtools index $i.sorted.guppy.bam; done

### Visualization with IGV
The Integrated Genome Viewer(`igv`) is a versatile JAVA program for displaying genomic data 
After starting the program, the terminal is occupied like with Dotter and can only be fed with commands again when the IGV window has been closed.

In the IGV window we first specify the reference genome in the menu item **Genomes > Load Genomes from File ...**. In our humble case it is the plasmid pBR322, the sequence of which is stored in the FASTA file *pBR322.fasta*. We then load the reads of the three experiments in BAM format (*bc0[123].all.fastq.mapped.ont.sam.sorted.bam*) in the menu **File > Load from File ...** With each When you load a data record, the window fills up. For each BAM file (or barcode or experiment) we see the reads and, above them, the hit density (coverage) that these reads generate. Don't be surprised if things look a little different for you.

![igv](igv.png)

It is noticeable that in the upper data set we see a gap at 670 base pairs. In the middle data set there is a gap at around 3,760 base pairs. In the lower data set, however, the entire plasmid sequence is covered by reads. 
What does that tell us? In the

- upper data set was the plasmid was cut once,
- twice in the middle data set (Cas9 and PstI),
- and in the lower data set not cut at all (undigested control).

Remember that reads cannot lie across interfaces. In the upper data set we find an exception: one read spans the interface at 670 bp, which may indicate a partially incomplete digestion
In the analysis presented, we only used the first thousand reads, which were saved by the MinION on the connected computer after around ten minutes of sequencing. That's a fraction of all data.